
Demonstrates molecular dynamics with constant energy.
<br>
from ase import units<br>
from ase.lattice.cubic import FaceCenteredCubic<br>
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution<br>
from ase.md.verlet import VelocityVerlet<br>
from ase.io.trajectory import Trajectory<br>
from ase.optimize.bfgs import BFGS<br>
from ase.constraints import StrainFilter<br>
# Use Asap for a huge performance increase if it is installed<br>
use_asap = True<br>
if use_asap:<br>
    from asap3 import EMT<br>
    size = 10<br>
else:<br>
    from ase.calculators.emt import EMT<br>
    size = 3<br>
# Set up a crystal<br>
atoms = FaceCenteredCubic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],<br>
                          symbol="Cu",<br>
                          size=(size, size, size),<br>
                          pbc=True)<br>
# Describe the interatomic interactions with the Effective Medium Theory<br>
atoms.calc = EMT()<br>
# Before we run the molecular dynamics, let's first optimise the unit cell<br>
constraints = StrainFilter(atoms)<br>
opt = BFGS(constraints, trajectory='opt.traj')<br>
opt.run(fmax=0.01)<br>
# Set the momenta corresponding to T=300K<br>
MaxwellBoltzmannDistribution(atoms, temperature_K=300)<br>
# We want to run MD with constant energy using the VelocityVerlet algorithm.<br>
dyn = VelocityVerlet(atoms, 5 * units.fs)  # 5 fs time step.<br>
def printenergy(a=atoms):  # store a reference to atoms in the definition.<br>
  
Function to print the potential, kinetic and total energy.


In [ ]:
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))

Now run the dynamics

In [ ]:
dyn.attach(printenergy, interval=10)

In [ ]:
traj = Trajectory('MD.traj', 'w', atoms)
dyn.attach(traj.write, interval=10)

In [ ]:
printenergy()
dyn.run(200)

Great! We've now successfully run an MD simulation with ASE!!!

Now it's your turn!

Run some MD simulations on other common metals: Al, Au, Ag, Pt, Pd